In [1]:
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
DATA_PATH = Path("~/data/split-v3-random-nongray").expanduser() 
BASE_PATH = Path("~/models/model-from-scratch-dropout-earlier/").expanduser()

In [3]:
model = load_model(str(BASE_PATH / "export"))

In [11]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# with (BASE_PATH / "export-tflite.tflite").open('wb') as f:
#     f.write(tflite_model)

## Post-training quantization

In [4]:
import numpy as np

In [5]:
mean = tf.Variable([[[0.74136317, 0.73086095, 0.74473]]])
IMAGE_SIZE = 336

In [6]:
# A generator that provides a representative dataset
def representative_data_gen(sample_size=1000):
    sample_paths = np.random.choice(list((DATA_PATH / "train").glob("*/*.jpeg")), 
                                    size=sample_size)
    
    for img_path in sample_paths:
        image = tf.io.read_file(str(img_path))
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image / 128., tf.float32) - mean
        image = tf.expand_dims(image, 0)
        yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# And this sets the representative dataset so we can quantize the activations
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with (BASE_PATH / "export-tflite_quant.tflite").open('wb') as f:
    f.write(tflite_model)

In [7]:
IMAGE_SIZE = 336

BATCH_SIZE = 90
RESCALE = 1/128

# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=RESCALE, 
#     featurewise_center=True,
#     width_shift_range=.025,
#     height_shift_range=.025,
#     brightness_range=(.8, 1.2),
#     zoom_range=.025,
# )

# train_generator = train_datagen.flow_from_directory(
#     DATA_PATH / "train",
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=RESCALE,
    featurewise_center=True
)
val_datagen.mean = mean

val_generator = val_datagen.flow_from_directory(
    DATA_PATH / "valid",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

Found 3280 images belonging to 2 classes.


In [9]:
batch_images, batch_labels = next(val_generator)

logits = model(batch_images)
prediction = np.argmax(logits, axis=1)
truth = np.argmax(batch_labels, axis=1)

keras_accuracy = tf.keras.metrics.Accuracy()
keras_accuracy(prediction, truth)

print("Raw model accuracy: {:.3%}".format(keras_accuracy.result()))

Raw model accuracy: 97.778%


In [59]:
def set_input_tensor(interpreter, input):
    input_details = interpreter.get_input_details()[0]
    tensor_index = input_details['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = input
    # NOTE: This model uses float inputs, but if inputs were uint8,
    # we would quantize the input like this:
    #   scale, zero_point = input_details['quantization']
    #   input_tensor[:, :] = np.uint8(input / scale + zero_point)

def classify_image(interpreter, input):
    set_input_tensor(interpreter, input)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = interpreter.get_tensor(output_details['index'])
    # NOTE: This model uses float outputs, but if outputs were uint8,
    # we would dequantize the results like this:
    #   scale, zero_point = output_details['quantization']
    #   output = scale * (output - zero_point)
    top_1 = np.argmax(output)
    return output[0][0], top_1

In [60]:
%%time 

interpreter = tf.lite.Interpreter(str(BASE_PATH / "export-tflite_quant.tflite"))
interpreter.allocate_tensors()

# Collect all inference predictions in a list
batch_prediction = []
batch_truth = np.argmax(batch_labels, axis=1)

for i in range(len(batch_images)):
    prob, klass = classify_image(interpreter, batch_images[i])
    batch_prediction.append(prediction)

# Compare all predictions to the ground truth
tflite_accuracy = tf.keras.metrics.Accuracy()
tflite_accuracy(batch_prediction, batch_truth)
print("Quant TF Lite accuracy: {:.3%}".format(tflite_accuracy.result()))

Quant TF Lite accuracy: 85.556%
CPU times: user 3min 50s, sys: 25.4 ms, total: 3min 50s
Wall time: 3min 50s


In [52]:
sample_paths = np.random.choice(list((DATA_PATH / "train").glob("*/*.jpeg")), 10)

In [58]:
import matplotlib.pyplot as plt
import cv2
KERAS_IMG_SIZE = (336, 336)
KERAS_RESCALE = 1/128
KERAS_MEAN = np.array([[[0.74136317, 0.73086095, 0.74473]]], dtype=np.float32)

img_path = sample_paths[3]
img = plt.imread(img_path)
img_small = cv2.resize(img, KERAS_IMG_SIZE, interpolation=cv2.INTER_AREA)
img_small_norm = (img_small * KERAS_RESCALE) - KERAS_MEAN
prob, klass = classify_image(interpreter, img_small_norm)
label = img_path.parent.name
klass_str = "BirdHome" if klass == 0 else "BirdRoaming"
print(f"Label: {img_path.parent.name}, pred: {klass_str}, prob: {prob:.3f}")

BirdHome
Label: BirdHome, pred: BirdHome, prob: 0.996


In [20]:
keras_accuracy.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.9777778>

In [17]:
tflite_accuracy.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.9777778>

In [15]:
! edgetpu_compiler $BASE_PATH/export-tflite_quant.tflite

Edge TPU Compiler version 2.1.302470888

Model compiled successfully in 352 ms.

Input model: /home/jvlier/models/model-from-scratch-dropout-earlier/export-tflite_quant.tflite
Input size: 410.00KiB
Output model: export-tflite_quant_edgetpu.tflite
Output size: 644.80KiB
On-chip memory used for caching model parameters: 79.50KiB
On-chip memory remaining for caching model parameters: 5.42MiB
Off-chip memory used for streaming uncached model parameters: 361.12KiB
Number of Edge TPU subgraphs: 1
Total number of operations: 13
Operation log: export-tflite_quant_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 11
Number of operations that will run on CPU: 2
See the operation log file for individual